In [50]:
import numpy as np
import plotly.graph_objects as go

In [51]:
def afficher_maillage_3D(X : np.array, Y : np.array, Z : np.array, titre : str, grandeur = None, nom_grandeur  = None, unite = None, afficher_array = None) -> None:
    precision_slider = 50 

    couleur = "temps" 
    methode_affichage = 'vscode' 
     
    #Gère le cas d'une valeur affectée aux points.
    if grandeur is not None:
        fig = go.Figure()
        mask_charge_positive = grandeur.flatten() > 0 #Filtre les valeurs positives.
        mask_charge_negative = grandeur.flatten() <= 0 #Filtre les valeurs négatives ou nulles.
        
        #Création d'une trace affichant seulement les grandeurs positives. C'est la trace(1)
        fig.add_trace(go.Scatter3d(
            x=X.flatten()[mask_charge_positive], y=Y.flatten()[mask_charge_positive], z=Z.flatten()[mask_charge_positive],
            mode="markers",
            marker=dict(size=2, #Taille des points.
                        color=grandeur.flatten()[mask_charge_positive],
                        colorscale=couleur,
                        cmin = grandeur.min(),
                        cmax = grandeur.max(),
                        opacity=1, 
                        colorbar=dict(
                        title=f'{nom_grandeur} ({unite})', #Titre de la colorbar 
                        ticks="outside",  
                        tickvals=[grandeur.min(), grandeur.max()], #Plage des valeurs de la colorbar
                        ticktext=[f"{grandeur.min():{trouver_signe_afficher_maillage_3D(grandeur.min())}.2f}", f"{grandeur.max():{trouver_signe_afficher_maillage_3D(grandeur.max())}.2f}"],
                    )),
            showlegend = False #Evite d'avoir "Trace 0" etc. affichées à droite.
        ))
        #Création d'une trace affichant seulement les grandeurs négatives. C'est la trace(2)
        fig.add_trace(go.Scatter3d(
            x=X.flatten()[mask_charge_negative], y=Y.flatten()[mask_charge_negative], z=Z.flatten()[mask_charge_negative],
            mode="markers",
            marker=dict(size=2, #Taille des points.
                        color=grandeur.flatten()[mask_charge_negative],
                        colorscale=couleur,
                        cmin = grandeur.min(),
                        cmax = grandeur.max(),
                        opacity=1,
                        colorbar=dict(
                        title=f'{nom_grandeur} ({unite})', #Titre de la colorbar 
                        ticks="outside",  
                        tickvals=[grandeur.min(), grandeur.max()], #Plage des valeurs de la colorbar
                        ticktext=[f"{grandeur.min():{trouver_signe_afficher_maillage_3D(grandeur.min())}.2f}", f"{grandeur.max():{trouver_signe_afficher_maillage_3D(grandeur.max())}.2f}"],  
                    )),
            showlegend= False #Evite d'avoir "Trace 0" etc. affichées à droite.
        ))
        
        #Création de len(np.linspace(grandeur.min(),grandeur.max(),precision_slider)) traces affichant seulement une gamme de grandeur. Ce sont
        #les trace(3), ..., trace(len(np.linspace(grandeur.min(),grandeur.max(),precision_slider)) + 2).
        tolerance = float((grandeur.max() - grandeur.min())/(2*precision_slider))
        for valeur in np.linspace(grandeur.min(),grandeur.max(),precision_slider):
            fig.add_trace(go.Scatter3d(
            x=X.flatten()[abs(grandeur.flatten()-valeur) <= tolerance], y=Y.flatten()[abs(grandeur.flatten()-valeur) <= tolerance], z=Z.flatten()[abs(grandeur.flatten()-valeur) <= tolerance],
            mode="markers",
            marker=dict(size=2, #Taille des points.
                        color=grandeur.flatten()[abs(grandeur.flatten()-valeur) <= tolerance],
                        colorscale=couleur,
                        cmin = grandeur.min(),
                        cmax = grandeur.max(),
                        opacity=1,
                        ),
            showlegend= False #Evite d'avoir "Trace 0" etc. affichées à droite.
        ))
        
        #Création d'un slider permettant de mettre en avant un certain intervalle de valeurs.
        steps = []
        for valeur, charge in zip(range(precision_slider),np.linspace(grandeur.min(),grandeur.max(),precision_slider)):
            step = dict(
                method='update',
                args=[
                    {'marker.opacity': calcul_liste_arg_slider_afficher_maillage_3D(valeur,precision_slider)}, #Modifie l'opacité de la trace(1), ..., trace(max).
                ],
                label = f" {charge:.2f}{chr(177)}{tolerance:.3f} {unite}" #Légende du slider.
            )
            steps.append(step)
        slider = [dict(
            active = int(precision_slider/2), #Position intiale du slider.
            currentvalue = {'prefix' : f'Valeur {nom_grandeur} :'}, #Nom du slider.
            pad = {'t':50},
            steps = steps
        )]
        
        #Création d'un bouton permettant de réafficher toutes les valeurs.
        liste_arg_reset_button = [1,1] #Respectivement l'opacité des charges positives puis négatives.
        for _ in range(precision_slider):
            liste_arg_reset_button.append(0) #La valeur ajoutée dans la liste est l'opacité des traces des gammes des valeurs.
        
        reset_button = dict(
            type='buttons',
            showactive=True,
            buttons=[
                dict(label='Réinitialiser Opacité', #Nom du bouton.
                     method='update',
                     args=[{'marker.opacity': liste_arg_reset_button}] #Modifie l'opacité de la trace(1), ..., trace(max).
                )
            ],
            x=0.1, #Position en %
            y=0.9, #Position en % 
            xanchor="right",
            yanchor="bottom"
        )
        
        #Création d'un bouton qui permet de mettre en avant les valeurs positives.   
        liste_arg_bouton_valeur_positive = [1,0.1] #Respectivement l'opacité des charges positives puis négatives.
        for _ in range(precision_slider):
            liste_arg_bouton_valeur_positive.append(0) #La valeur ajoutée dans la liste est l'opacité des traces des gammes des valeurs.
     
        bouton_valeur_positive = dict(
            type='buttons',
            showactive=True,
            buttons=[
                dict(label=f'Afficher {nom_grandeur} +', #Nom du bouton.
                     method='update',
                     args=[{'marker.opacity': liste_arg_bouton_valeur_positive}] #Modifie l'opacité de la trace(1), ..., trace(max).
                )
            ],
            x=0.1, #Position en %
            y=0.7, #Position en %
            xanchor="right",
            yanchor="bottom"
        )
        
        #Création d'un bouton qui permet de mettre en avant les valeurs positives.   
        liste_arg_bouton_valeur_negative = [0.1,1] #Respectivement l'opacité des charges positives puis négatives.
        for _ in range(precision_slider):
            liste_arg_bouton_valeur_negative.append(0) #La valeur ajoutée dans la liste est l'opacité des traces des gammes des valeurs.
            
        bouton_valeur_negative = dict(
            type='buttons',
            showactive=True,
            buttons=[
                dict(label=f'Afficher {nom_grandeur} -', #Nom du bouton.
                     method='update',
                     args=[{'marker.opacity': liste_arg_bouton_valeur_negative}] #Modifie l'opacité de la trace(1), ..., trace(max).
                )
            ],
            x=0.1, #Position en %
            y=0.5, #Position en %
            xanchor="right",
            yanchor="bottom"
        )
        #Update et affiche le plot selon methode_affichage.
        fig.update_layout(
            title = titre,
            sliders = slider,
            updatemenus = [reset_button,bouton_valeur_positive,bouton_valeur_negative],
            scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z")
        )
        fig.show(methode_affichage)
    #Gère le cas dans lequel aucune valeur n'est affectée aux points.
    else:
        #Création d'une trace contenant tous les points du maillage.
        fig = go.Figure(data=[go.Scatter3d(
            x=X.flatten(), y=Y.flatten(), z=Z.flatten(),
            mode="markers",
            marker=dict(size=2, opacity=0.8) 
        )])
        #Update et affiche le plot selon methode_affichage.
        fig.update_layout(
            title=titre,
            scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z")
        )
        fig.show(methode_affichage)
    #Gère le cas où l'on souhaite voir à quoi ressemblent les matrices du maillage X,Y,Z.
    if afficher_array:
        print(f"La première matrice ressemble à :\n\n{X},\n\n"
              f"La deuxième matrice ressemble à :\n\n{Y},\n\n"
              f"La dernière matrice ressemble à :\n\n{Z}")
    
    #C'est une méthode, elle ne retourne rien mais permet seulement d'afficher.
    return None

In [52]:
def creer_maillage_3D(x_min : float, x_max : float, y_min : float, y_max : float, z_min : float, z_max : float, pas_du_maillage : float) -> list:
    for min, max in zip([x_min,y_min,z_min],[x_max,y_max,z_max]):
        if min == max: 
            raise ValueError(f"Le maillage n'est pas 3D: {min} = {max}.")
        if min > max:
            min, max = max, min 
        if pas_du_maillage > float(abs(max-min)/2):
            raise ValueError("pas_du_maillage est trop grand.") 

    longueur_maillage_x = abs(x_max - x_min)
    longueur_maillage_y = abs(y_max - y_min)
    longueur_maillage_z = abs(z_max - z_min)
    
    nb_points_entre_x_min_x_max = int((longueur_maillage_x) / (pas_du_maillage)) + 1
    nb_points_entre_y_min_y_max = int((longueur_maillage_y) / (pas_du_maillage)) + 1
    nb_points_entre_z_min_z_max = int((longueur_maillage_z) / (pas_du_maillage)) + 1

    axe_x = np.linspace(x_min, x_max, nb_points_entre_x_min_x_max)
    axe_y = np.linspace(y_min, y_max, nb_points_entre_y_min_y_max)
    axe_z = np.linspace(z_min, z_max, nb_points_entre_z_min_z_max)
    
    X,Y,Z = np.meshgrid(axe_x,axe_y,axe_z)

    X = np.round(X, 1) 
    Y = np.round(Y, 1)
    Z = np.round(Z, 1)

    return [X,Y,Z]

cote = 1
pas_du_maillage = 1
X,Y,Z = creer_maillage_3D(-cote,cote,-cote,cote,-cote,cote,pas_du_maillage)

afficher_maillage_3D(X,Y,Z,"Visualisation du maillage")

P1 = np.array([X[0],Y[0],Z[0]])

In [53]:
sphere = lambda r, theta, phi: np.array([r * np.sin(theta) * np.cos(phi), r * np.sin(theta) * np.sin(phi), r * np.cos(theta)])
cylindre = lambda r, theta, z: np.array([r * np.cos(theta), r * np.sin(theta), z])

In [54]:
n = 50
r = 10 * np.ones((n, n))  
theta = np.linspace(0, np.pi, n)
phi = np.linspace(0, 2*np.pi, n) 

Theta, Phi = np.meshgrid(theta, phi)

X, Y, Z = sphere(r, Theta, Phi)

x_flat = X.flatten()
y_flat = Y.flatten()
z_flat = Z.flatten()

fig = go.Figure(data=[go.Scatter3d(x=x_flat, y=y_flat, z=z_flat, mode='markers',marker=dict(size=2))])
fig.show()

In [55]:
P1

array([[[-1., -1., -1.],
        [ 0.,  0.,  0.],
        [ 1.,  1.,  1.]],

       [[-1., -1., -1.],
        [-1., -1., -1.],
        [-1., -1., -1.]],

       [[-1.,  0.,  1.],
        [-1.,  0.,  1.],
        [-1.,  0.,  1.]]])

In [56]:
X,Y,Z

(array([[0.00000000e+00, 6.40702200e-01, 1.27877162e+00, ...,
         1.27877162e+00, 6.40702200e-01, 1.22464680e-15],
        [0.00000000e+00, 6.35442044e-01, 1.26827292e+00, ...,
         1.26827292e+00, 6.35442044e-01, 1.21459247e-15],
        [0.00000000e+00, 6.19747947e-01, 1.23694922e+00, ...,
         1.23694922e+00, 6.19747947e-01, 1.18459456e-15],
        ...,
        [0.00000000e+00, 6.19747947e-01, 1.23694922e+00, ...,
         1.23694922e+00, 6.19747947e-01, 1.18459456e-15],
        [0.00000000e+00, 6.35442044e-01, 1.26827292e+00, ...,
         1.26827292e+00, 6.35442044e-01, 1.21459247e-15],
        [0.00000000e+00, 6.40702200e-01, 1.27877162e+00, ...,
         1.27877162e+00, 6.40702200e-01, 1.22464680e-15]]),
 array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  8.19311788e-02,  1.63525685e-01, ...,
          1.63525685e-01,  8.19311788e-02,  1.56604357e-16],
        [ 0.0

In [57]:
#x_flat,y_flat,z_flat 
#np.sqrt(x_flat**2 + y_flat**2 + z_flat**2)

In [58]:
n = 50
theta = np.linspace(0, 2*np.pi, n)
z = np.linspace(-1, 1, n) # 10 * np.ones((n, n)) 
r = z * np.ones((n, n))   # np.linspace(0, 20, n) 

Theta,Z = np.meshgrid(theta,z)


X, Y, Z = cylindre(r.T, Theta, Z)

x_flat = X.flatten()
y_flat = Y.flatten()
z_flat = Z.flatten()

fig = go.Figure(data=[go.Scatter3d(x=x_flat, y=y_flat, z=z_flat, mode='markers',marker=dict(size=2))])
fig.show()

In [59]:
#x_flat,y_flat,z_flat; np.sqrt(x_flat**2 + y_flat**2 + z_flat**2)